In [13]:
import tensorflow as tf
import tf2onnx
from tensorflow.keras.applications.resnet50 import ResNet50
import bentoml
model = ResNet50(weights='imagenet')

In [14]:
spec = (tf.TensorSpec((None, 224, 224, 3), tf.float32, name="input"),)
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13)

2022-08-26 13:47:40.537114: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-08-26 13:47:40.537367: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-08-26 13:47:43.697234: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-08-26 13:47:43.697506: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session


In [15]:
signatures = {
    "run": {"batchable": True},
}
bentoml.onnx.save_model("onnx_resnet50", onnx_model, signatures=signatures)

Model(tag="onnx_resnet50:wgoyv7rfiwwucntc", path="/home/admin/bentoml/models/onnx_resnet50/wgoyv7rfiwwucntc/")

In [16]:
%%writefile service.py
import bentoml

import numpy as np
from bentoml.io import Image
from bentoml.io import JSON

runner = bentoml.onnx.get("onnx_resnet50:latest").to_runner()

svc = bentoml.Service("onnx_resnet50", runners=[runner])

@svc.api(input=Image(), output=JSON())
def predict(img):

    from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

    img = img.resize((224, 224))
    arr = np.array(img)
    arr = np.expand_dims(arr, axis=0)
    arr = preprocess_input(arr)
    preds = runner.run.run(arr)
    return decode_predictions(preds, top=1)[0]

Overwriting service.py


In [17]:
!bentoml build

Building BentoML service "onnx_resnet50:wjyjlnrfiwzr4ntc" from build context "/home/admin/notebooks/bentoml/resnet50"
Packing model "onnx_resnet50:wgoyv7rfiwwucntc"

██████╗░███████╗███╗░░██╗████████╗░█████╗░███╗░░░███╗██╗░░░░░
██╔══██╗██╔════╝████╗░██║╚══██╔══╝██╔══██╗████╗░████║██║░░░░░
██████╦╝█████╗░░██╔██╗██║░░░██║░░░██║░░██║██╔████╔██║██║░░░░░
██╔══██╗██╔══╝░░██║╚████║░░░██║░░░██║░░██║██║╚██╔╝██║██║░░░░░
██████╦╝███████╗██║░╚███║░░░██║░░░╚█████╔╝██║░╚═╝░██║███████╗
╚═════╝░╚══════╝╚═╝░░╚══╝░░░╚═╝░░░░╚════╝░╚═╝░░░░░╚═╝╚══════╝

Successfully built Bento(tag="onnx_resnet50:wjyjlnrfiwzr4ntc")
